In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [2]:
df = pd.read_csv('../data/occupation_shares.csv')
df = df.iloc[:-1]
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)
df = df.set_index('MET2013')

In [3]:
# def get_shifter(pi_c, pi_c_prime, rho_k=0.5):
#     return (pi_c / pi_c_prime)**(1 - rho_k)

# # Assuming df is your DataFrame
# df_new = df.copy()
# df_new = df_new.iloc[1:]

# for column in df_new.columns:
#     pi_c_prime = df_new.iloc[0][column]
#     df_new[column] = df_new[column].apply(lambda x: get_shifter(x, pi_c_prime))
# df_new.to_csv('shifter_estimates.csv')

In [4]:
df_new = df.copy()
df_new = df_new.iloc[1:]
df_new.drop(columns=['other', 'city_total'], inplace=True)
# df_new = df_new.div(df_new.iloc[0], axis='columns')
data_flat = df_new.stack().reset_index()
data_flat.columns = ['city', 'occupation', 'value']
data_dummies = pd.get_dummies(data_flat, columns=['occupation', 'city'], drop_first=True)

X = data_dummies.drop('value', axis=1)
y = data_dummies['value']
model = sm.OLS(y, X).fit()

coef = model.params
occupation_coef = coef[coef.index.str.startswith('occupation_')]
city_coef = coef[coef.index.str.startswith('city_')]
occupation_coef = pd.DataFrame(occupation_coef)
occupation_coef.columns = ['coef']
occupation_coef.index = occupation_coef.index.str.replace('occupation_', '')
city_coef = pd.DataFrame(city_coef)
city_coef.index = city_coef.index.str.replace('city_', '')
city_coef.columns = ['coef']

occupation_coef.to_csv('occupation_coef.csv')
city_coef.to_csv('city_coef.csv')

In [5]:
resid = model.resid
data_dummies['resid'] = resid
resid = data_dummies['resid'].values
resid = resid.reshape(df_new.shape[0], df_new.shape[1])
resid = pd.DataFrame(resid, index=df_new.index, columns=df_new.columns)
resid['city_total'] = resid.sum(axis=1)
resid.to_csv('residuals.csv')

In [6]:
ammenities = resid['city_total']
ammenities = pd.DataFrame(ammenities)
ammenities['city_coef'] = city_coef['coef']
ammenities['shifter'] = ammenities['city_coef'] / ammenities['city_total']
ammenities['shifter_normalized'] = ammenities['shifter'] ** 2 
ammenities['shifter_normalized'] = ammenities['shifter_normalized'] / ammenities['shifter_normalized'][1]
ammenities.to_csv('ammenities.csv')

/tmp/ipykernel_284007/1977351028.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ammenities['shifter_normalized'] = ammenities['shifter_normalized'] / ammenities['shifter_normalized'][1]
